# Capstone Project - Loan Classification

## Background
Peer to peer lending, or P2P lending, has been around for more than a decade. Loans are made out to a variety of borrowers for a variety of 

## Import libraries and data

### Import libraries
Pandas and Numpy are the libraries that will be imported at this stage. This is because, at this point, the tasks in this notebook will be to import the data and process the data so as to make it suitable for creating models

In [1]:
# Import libraries
import pandas as pd # For placing the data into dataframes for analysis
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 160)
pd.set_option('display.max_rows', 400)

### Import and review the data
The sourced data from The Lending Club will be loaded into the notebook and reviewed before doing the necessary amendments to the data

In [2]:
accepted = pd.read_csv('../assets/accepted_2007_to_2018Q4.csv', low_memory = False)

In [3]:
# Quick look at the data imported
accepted.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,N

In [4]:
accepted.dtypes

id                                             object
member_id                                     float64
loan_amnt                                     float64
funded_amnt                                   float64
funded_amnt_inv                               float64
term                                           object
int_rate                                      float64
installment                                   float64
grade                                          object
sub_grade                                      object
emp_title                                      object
emp_length                                     object
home_ownership                                 object
annual_inc                                    float64
verification_status                            object
issue_d                                        object
loan_status                                    object
pymnt_plan                                     object
url                         

In [5]:
accepted.shape

(2260701, 151)

## Data Cleaning

### Period under consideration

Based on the delinquency rates as seen in the graph below, there was about spike in loan delinquency for the years of 2009 and 2010.

This can be largely attributable to the financial crisis before that. Subsequently, in the years that follow, the delinquency rates gradually fell. The last 2 years of the data, 2017 and 2019 will be used as the test data. Data used for modeling will be the preceding 3 years. The rationale behind this is that the period from 2009 and 2010 coincided with the financial crisis. So taking into account those years might distort the risk involved. Therefore, choosing 3 preceding years might give the model training data with a closer semblence to the test period.

In [6]:
# Change the format of the issue date column to a date time format to only take data from 2014
accepted['issue_d'] = pd.to_datetime(accepted['issue_d'])
# accepted['earliest_cr_line'] = pd.to_datetime(accepted['earliest_cr_line'])

In [7]:
# Take the latest date available in the data
accepted.issue_d.max()

Timestamp('2018-12-01 00:00:00')

In [8]:
# Take the earliest date available in the data
accepted.issue_d.min()

Timestamp('2007-06-01 00:00:00')

In [9]:
# Only take data from the latest 5 years in the data
accepted = accepted[accepted['issue_d'].dt.year > 2013]

In [10]:
# Check data to ensure that it only holds from 2014 onwards
accepted.issue_d.min()

Timestamp('2014-01-01 00:00:00')

In [11]:
accepted.shape

(2029952, 151)

It would seem that the data has been loaded without any issues. The next few steps would involve reviewing the data to see
* What columns and/or rows should be dropped
    * Too many null values
    * Leak data from future
* Which cell values need to be updated
* Which rows need to have data imputed
* Any duplicates

Based on the descriptions from the given data dictionary, the features in the table below will not be known to an investor/lender as they are retrospective. If they are included in the model, they will expose 'hindsight' information. This will not be in line with the reality of the situation. Hence, they will be dropped from the data.

In [12]:
hindsight_cols = ['out_prncp',
'out_prncp_inv',
'total_pymnt',
'total_pymnt_inv',
'total_rec_prncp',
'total_rec_int',
'total_rec_late_fee',
'recoveries',
'collection_recovery_fee',
'last_pymnt_d',
'last_pymnt_amnt',
'next_pymnt_d',
'last_credit_pull_d',
'collections_12_mths_ex_med',
'acc_now_delinq',
'tot_coll_amt',
'tot_cur_bal',
'open_acc_6m',
'open_act_il',
'open_il_12m',
'open_il_24m',
'mths_since_rcnt_il',
'total_bal_il',
'il_util',
'open_rv_12m',
'open_rv_24m',
'max_bal_bc',
'all_util',
'total_rev_hi_lim',
'avg_cur_bal',
'chargeoff_within_12_mths',
'delinq_amnt',
'mths_since_recent_bc_dlq',
'mths_since_recent_inq',
'mths_since_recent_revol_delinq',
'num_accts_ever_120_pd',
'num_tl_120dpd_2m',
'num_tl_30dpd',
'num_tl_90g_dpd_24m',
'num_tl_op_past_12m',
'hardship_flag',
'hardship_type',
'hardship_reason',
'hardship_status',
'deferral_term',
'hardship_amount',
'hardship_start_date',
'hardship_end_date',
'payment_plan_start_date',
'hardship_length',
'hardship_dpd',
'hardship_loan_status',
'orig_projected_additional_accrued_interest',
'hardship_payoff_balance_amount',
'hardship_last_payment_amount',
'debt_settlement_flag',
'debt_settlement_flag_date',
'settlement_status',
'settlement_date',
'settlement_amount',
'settlement_percentage',
'settlement_term']

In [13]:
accepted.drop(columns = hindsight_cols, inplace = True)

In [14]:
accepted.shape

(2029952, 89)

In [15]:
# Drop columns with all null values
accepted.dropna(axis = 1, how = 'all', inplace = True)

In [16]:
accepted.shape

(2029952, 88)

In [19]:
accepted.isnull().sum()

id                                           0
loan_amnt                                    0
funded_amnt                                  0
funded_amnt_inv                              0
term                                         0
int_rate                                     0
installment                                  0
grade                                        0
sub_grade                                    0
emp_title                               152606
emp_length                              137908
home_ownership                               0
annual_inc                                   0
verification_status                          0
issue_d                                      0
loan_status                                  0
pymnt_plan                                   0
url                                          0
desc                                   2014605
purpose                                      0
title                                    23305
zip_code     

In [20]:
# Following columns are to be dropped due to no perceived impact and/or too many null values to be meaningful
accepted.drop(columns = ['emp_title',
                        'url',
                        'title',
                        'zip_code',
                        ],
              inplace = True)

In [22]:
accepted[['dti',
          'inq_last_6mths',
          'mths_since_last_delinq',
          'mths_since_last_record',
          'revol_util',
          'mths_since_last_major_derog',
          'annual_inc_joint',
          'dti_joint',
          'inq_fi',
          'total_cu_tl',
          'inq_last_12m',
          'bc_open_to_buy',
          'bc_util',
          'mo_sin_old_il_acct',
          'mths_since_recent_bc',
          'num_rev_accts',
          'pct_tl_nvr_dlq',
          'emp_length',
          'percent_bc_gt_75']] = accepted[['dti',
          'inq_last_6mths',
          'mths_since_last_delinq',
          'mths_since_last_record',
          'revol_util',
          'mths_since_last_major_derog',
          'annual_inc_joint',
          'dti_joint',
          'inq_fi',
          'total_cu_tl',
          'inq_last_12m',
          'bc_open_to_buy',
          'bc_util',
          'mo_sin_old_il_acct',
          'mths_since_recent_bc',
          'num_rev_accts',
          'pct_tl_nvr_dlq',
          'emp_length',
          'percent_bc_gt_75']].fillna(0)

In [23]:
# List down columns which have more than 80% of null values
null_col = accepted.columns[accepted.isnull().mean() > 0.8]
len(null_col)

15

In [24]:
# Drop columns based on the list in null_col, which are those with more than 80% null values
accepted.drop(columns = null_col, inplace = True)

In [25]:
accepted.shape

(2029952, 69)

In [26]:
accepted.isnull().sum()

id                             0
loan_amnt                      0
funded_amnt                    0
funded_amnt_inv                0
term                           0
int_rate                       0
installment                    0
grade                          0
sub_grade                      0
emp_length                     0
home_ownership                 0
annual_inc                     0
verification_status            0
issue_d                        0
loan_status                    0
pymnt_plan                     0
purpose                        0
addr_state                     0
dti                            0
delinq_2yrs                    0
earliest_cr_line               0
fico_range_low                 0
fico_range_high                0
inq_last_6mths                 0
mths_since_last_delinq         0
mths_since_last_record         0
open_acc                       0
pub_rec                        0
revol_bal                      0
revol_util                     0
total_acc 

In [27]:
accepted.policy_code.unique()

array([1.])

In [28]:
accepted.drop('policy_code', axis = 1, inplace = True)

### Data types
This is to see if the data types are according to the expected types. This is for analysis

In [29]:
accepted.dtypes

id                                     object
loan_amnt                             float64
funded_amnt                           float64
funded_amnt_inv                       float64
term                                   object
int_rate                              float64
installment                           float64
grade                                  object
sub_grade                              object
emp_length                             object
home_ownership                         object
annual_inc                            float64
verification_status                    object
issue_d                        datetime64[ns]
loan_status                            object
pymnt_plan                             object
purpose                                object
addr_state                             object
dti                                   float64
delinq_2yrs                           float64
earliest_cr_line                       object
fico_range_low                    

Employment length goes some way towards 

In [30]:
accepted.emp_length.unique()

array(['10+ years', '3 years', '4 years', '6 years', '1 year', '7 years',
       '8 years', '5 years', '2 years', '9 years', '< 1 year', 0],
      dtype=object)

In [31]:
# Create mapping for employment length
emp_length_dict = {'10+ years': 10,
                   '9 years': 9,
                   '8 years': 8,
                   '7 years': 7,
                   '6 years': 6,
                   '5 years': 5,
                   '4 years': 4,
                   '3 years': 3,
                   '2 years': 2,
                   '1 year': 1,
                   '< 1 year': 0}

In [32]:
# Map values to dictionary for employment length to convert to 
accepted = accepted.replace({'emp_length': emp_length_dict})

In [33]:
accepted['emp_length'] = accepted['emp_length'].fillna(0)

In [34]:
accepted.dtypes

id                                     object
loan_amnt                             float64
funded_amnt                           float64
funded_amnt_inv                       float64
term                                   object
int_rate                              float64
installment                           float64
grade                                  object
sub_grade                              object
emp_length                              int64
home_ownership                         object
annual_inc                            float64
verification_status                    object
issue_d                        datetime64[ns]
loan_status                            object
pymnt_plan                             object
purpose                                object
addr_state                             object
dti                                   float64
delinq_2yrs                           float64
earliest_cr_line                       object
fico_range_low                    

In [35]:
# This is to remove the spaces and the word 'months' from the 'term' column
accepted['term'] = accepted['term'].str.replace(' ', '')
accepted['term'] = accepted['term'].str.replace('months', '')
accepted['term'] = accepted['term'].astype('float64')

In [36]:
accepted.dtypes

id                                     object
loan_amnt                             float64
funded_amnt                           float64
funded_amnt_inv                       float64
term                                  float64
int_rate                              float64
installment                           float64
grade                                  object
sub_grade                              object
emp_length                              int64
home_ownership                         object
annual_inc                            float64
verification_status                    object
issue_d                        datetime64[ns]
loan_status                            object
pymnt_plan                             object
purpose                                object
addr_state                             object
dti                                   float64
delinq_2yrs                           float64
earliest_cr_line                       object
fico_range_low                    

In [ ]:
accepted.shape

In [ ]:
accepted.isnull().sum()

In [ ]:
accepted.shape

In [ ]:
accepted.isnull().sum()

In [ ]:
accepted.head()

In [ ]:
accepted.isnull().sum()

In [ ]:
accepted.term.unique()

In [ ]:
accepted.head()

In [ ]:
accepted.dtypes